In [1]:
import socket
import time
import os
os.chdir('..')
from Go.go import Go
from Attaxx.attaxx import Attaxx

Game=str(input("specify game and board dimensions:\n")) # "A6x6" "G7x7" "G9x9" "A5x5" "A4x4"

def start_server(current_game, host='localhost', port=12345):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(2)

    print("Waiting for two agents to connect...")
    agent1, addr1 = server_socket.accept()
    print("Agent 1 connected from", addr1)
    bs=b'AG1 '+Game.encode()
    agent1.sendall(bs)

    agent2, addr2 = server_socket.accept()
    print("Agent 2 connected from", addr2)
    bs=b'AG2 '+Game.encode()
    agent2.sendall(bs)

    agents = [agent1, agent2]
    current_agent = 0
    
    state = current_game.get_initial_state()
    print(state)

    while True:
        try:
            data = agents[current_agent].recv(1024).decode()
            if not data:
                break

            move = data
            print(state)
            print(f"Agent {agents[current_agent]} sent move {move}")
            
            winner, game_over = current_game.get_value_and_terminated(state, None, None)

            if game_over: 
                agents[current_agent].sendall(b'END')
                agents[1-current_agent].sendall(b'END')
                break
            
            if current_game.is_valid_move(state, move, agents[current_agent]):
                agents[current_agent].sendall(b'VALID')
                agents[1-current_agent].sendall(data.encode())
                new_state = current_game.get_next_state(state, move, agents[current_agent])
                print(new_state)
                state = new_state
            else:
                agents[current_agent].sendall(b'INVALID')

            # Switch to the other agent
            current_agent = 1-current_agent
            time.sleep(1)

        except Exception as e:
            print("Error:", e)
            break

    print("\n-----------------\nGAME END\n-----------------\n")
    time.sleep(1)
    agent1.close()
    agent2.close()
    server_socket.close()

if __name__ == "__main__":
    if Game[0] == 'A':
        current_game = Attaxx([int(Game[-1]), int(Game[-1])])
    elif Game[0] == 'G':
        current_game = Go([int(Game[-1]), int(Game[-1])], 5.5)
    start_server(current_game)

A5x5
True
Waiting for two agents to connect...
Agent 1 connected from ('127.0.0.1', 56334)
Agent 2 connected from ('127.0.0.1', 56336)
[[ 1.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  1.]]
[[ 1.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  1.]]
Agent <socket.socket fd=1444, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 12345), raddr=('127.0.0.1', 56334)> sent move 0001
Error: is_valid_move() missing 2 required positional arguments: 'action' and 'player'

-----------------
GAME END
-----------------

